## [Practice 6] Lidar Object Detection
* 0. 실습 환경 설정
* 1. 데이터셋 준비하기
* 2. Training
* 3. Testing
* 4. Inference
    * 4-1.Import modules / 하이퍼파라미터 설정
    * 4-2. 모델 불러오기
    * 4-3. 2D 및 BEV Detection결과 시각화

### 0. 실습 환경 설정
* 가상환경명 : lidar
* Python 버전 : python 3.8
* 설치 목록
    * !pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html
    * opencv_python==4.1.0.25
    * matplotlib
* Complex-YOLO Git 주소 : https://github.com/maudzung/Complex-YOLOv4-Pytorch

In [4]:
#!git clone https://github.com/maudzung/Complex-YOLOv4-Pytorch

Cloning into 'Complex-YOLOv4-Pytorch'...


### 1. 데이터셋 준비하기
* 이론 PPT에 나와있는 것 참고하여 다음과 같이 Folder structure 구성하기
```
${ROOT}
└── checkpoints/    
    ├── complex_yolov3/
    └── complex_yolov4/
└── dataset/    
    └── kitti/
        ├──ImageSets/
        │   ├── train.txt
        │   └── val.txt
        ├── training/
        │   ├── image_2/ <-- for visualization
        │   ├── calib/
        │   ├── label_2/
        │   └── velodyne/
        └── testing/  
        │   ├── image_2/ <-- for visualization
        │   ├── calib/
        │   └── velodyne/ 
        └── classes_names.txt
└── src/
    ├── config/
    ├── cfg/
        │   ├── complex_yolov3.cfg
        │   ├── complex_yolov3_tiny.cfg
        │   ├── complex_yolov4.cfg
        │   ├── complex_yolov4_tiny.cfg
    │   ├── train_config.py
    │   └── kitti_config.py
    ├── data_process/
    │   ├── kitti_bev_utils.py
    │   ├── kitti_dataloader.py
    │   ├── kitti_dataset.py
    │   ├── kitti_data_utils.py
    │   ├── train_val_split.py
    │   └── transformation.py
    ├── models/
    │   ├── darknet2pytorch.py
    │   ├── darknet_utils.py
    │   ├── model_utils.py
    │   ├── yolo_layer.py
    └── utils/
    │   ├── evaluation_utils.py
    │   ├── iou_utils.py
    │   ├── logger.py
    │   ├── misc.py
    │   ├── torch_utils.py
    │   ├── train_utils.py
    │   └── visualization_utils.py
    ├── evaluate.py
    ├── test.py
    ├── test.sh
    ├── train.py
    └── train.sh
├── README.md 
└── requirements.txt
```

### 1. Training

* TODO : yolov4.weights 다운로드 [링크](https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights)
* TODO : ${root}/checkpoints 에 yolov4.weights 넣어주기
* TODO : 경로 변경 (${root}/src 로 변경)

In [ ]:
cd

In [ ]:
!python train.py --gpu_idx 0  --pretrained_path ../checkpoints/yolov4.weights --cfgfile ./config/cfg/complex_yolov4.cfg --save_path ../checkpoints/Complex_yolo_yolo_v4.pth --num_epochs 300

### 2. Testing

* TODO : 경로 변경 (${root}/src 로 변경)
* TODO : pretrained_path에 해당 [링크](https://drive.google.com/drive/folders/1RHD9PBvk-9SjbKwoi_Q1kl9-UGFo2Pth)의 가중치나 학습한 가중치 경로 넣어주기

In [ ]:
cd

In [ ]:
!python test.py --gpu_idx 0 --pretrained_path ../checkpoints/complex_yolov4_mse_loss.pth --cfgfile ./config/cfg/complex_yolov4.cfg --save_test_output

### 3. Inference
### 3-1.Import modules / 하이퍼파라미터 설정

* TODO : 경로 변경 (${root}/src 로 변경)

In [ ]:
cd

In [ ]:
import cv2
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

from models.darknet2pytorch import Darknet
from data_process import kitti_data_utils,kitti_bev_utils
from utils.evaluation_utils import post_processing, rescale_boxes, post_processing_v2
from utils.visualization_utils import show_image_with_boxes, merge_rgb_to_bev, predictions_to_kitti_format
import config.kitti_config as cnf

* TODO : weight_path에 해당 [링크](https://drive.google.com/drive/folders/1RHD9PBvk-9SjbKwoi_Q1kl9-UGFo2Pth)의 가중치나 학습한 가중치 경로 넣어주기
* TODO : kitti dataset에서 0번만 빼서 ${root}/sample에 넣어주기
    * ${root}/dataset/kitti/training/image_2/000000.png
    * ${root}/dataset/kitti/training/label_2/000000.txt
    * ${root}/dataset/kitti/training/velodyne/000000.bin


In [3]:
rgb_path = '../sample/000000.png'
lidar_path = '../sample/000000.bin'
calib_path = '../sample/000000.txt'

cfg_file = './config/cfg/complex_yolov4.cfg'
weight_path = 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

nms_thresh =0.5
conf_thresh = 0.5
img_size = 608

### 3-2. 모델 불러오기

In [4]:
model = Darknet(cfgfile=cfg_file, use_giou_loss=False)
model.load_state_dict(torch.load(weight_path, map_location=device))
model = model.to(device=device)
model.eval()
print("model loaded!")

[INFO] No error, the convolution haven't activate linear
[INFO] No error, the convolution haven't activate linear
[INFO] No error, the convolution haven't activate linear
model loaded!


### 3-3. 2D 및 BEV Image Detection 결과 시각화

In [ ]:
lidar_data = np.fromfile(lidar_path, dtype=np.float32).reshape(-1, 4)
b = kitti_bev_utils.removePoints(lidar_data, cnf.boundary)
rgb_map = kitti_bev_utils.makeBVFeature(b, cnf.DISCRETIZATION, cnf.boundary)
rgb_map = torch.from_numpy(rgb_map).unsqueeze(0).float()
input_imgs = rgb_map.to(device=device).float()
outputs = model(input_imgs)
detections = post_processing_v2(outputs, conf_thresh=conf_thresh, nms_thresh=nms_thresh)

img_detections = []  # Stores detections for each image index
img_detections.extend(detections)

img_bev = rgb_map.squeeze() * 255
img_bev = img_bev.permute(1, 2, 0).numpy().astype(np.uint8)
img_bev = cv2.resize(img_bev, (img_size, img_size))
img_bev_v = img_bev.copy()

for detections in img_detections:
    if detections is None:
        continue
    # Rescale boxes to original image
    detections = rescale_boxes(detections, img_size, img_bev.shape[:2])
    for x, y, w, l, im, re, *_, cls_pred in detections:
        yaw = np.arctan2(im, re)
        # Draw rotated box
        kitti_bev_utils.drawRotatedBox(img_bev, x, y, w, l, yaw, cnf.colors[int(cls_pred)])

img_rgb = cv2.imread(rgb_path)
calib = kitti_data_utils.Calibration(calib_path)
objects_pred = predictions_to_kitti_format(img_detections, calib, img_rgb.shape, img_size)
img_rgb = show_image_with_boxes(img_rgb, objects_pred, calib, False)

img_bev_flip = cv2.flip(cv2.flip(img_bev, 0), 1)
out_img = merge_rgb_to_bev(img_rgb, img_bev_flip, output_width=608)